###  **Data preprocessing - MNL model** <br>

This notebook is used for the pre-processing of the data utilized in the MNL model. Unlike the data used in conjuction with the Random Forest model, NaN values in this case are not imputed.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_2018_2019_final = pd.read_csv('./data/data_final_2018_2019.csv', index_col = 0)

# get the parking costs data
parking_costs = pd.read_csv('./data/pc4_parking_avg_charge.csv')

In [3]:
data_2018_2019_final.khvm.value_counts() 

khvm
1    86211
5    52767
2    27127
3     8998
4     8039
6     7679
Name: count, dtype: int64

In [4]:
# add features in the dataset

# parking_costs (eur/h)
parking_costs.rename(columns = {'PC4':'aankpc'}, inplace = True)
data_2018_2019_final = pd.merge(data_2018_2019_final, parking_costs, on = 'aankpc')

# trips with purpose 'going home' are assumed to have no associated costs
data_2018_2019_final.loc[(data_2018_2019_final.doel == 1),'AVG_CHARGE'] = 0

# peak-hour (1: trip takes place during peak hours,0: trip does not take place during peak hours)
data_2018_2019_final.loc[(data_2018_2019_final.verttijd < '09:00') & (data_2018_2019_final.verttijd >= '06:30'),'peak_hour'] = 1
data_2018_2019_final.loc[(data_2018_2019_final.verttijd < '18:30') & (data_2018_2019_final.verttijd >= '16:00'),'peak_hour'] = 1
data_2018_2019_final.loc[data_2018_2019_final.peak_hour != 1, 'peak_hour'] = 0

# urbanity level of the origin and destination postcodes
sted_data = pd.read_csv('./data/sted_data.csv', sep = ',')
sted_data.rename(columns = {'PC4':'vertpc', 'sted':'sted_o'}, inplace = True)

data_2018_2019_final = pd.merge(data_2018_2019_final, sted_data, on = 'vertpc')

sted_data.rename(columns = {'vertpc':'aankpc','sted_o':'sted_d'}, inplace = True)
data_2018_2019_final = pd.merge(data_2018_2019_final, sted_data, on = 'aankpc')

In [5]:
# inspect NaN values
data_2018_2019_final.isnull().sum()

opid                 0
hhpers               0
hhsam                0
hhlft1               0
hhlft2               0
hhlft3               0
hhlft4               0
wopc                 0
wogem                0
sted                 0
gemgr                0
prov                 0
geslacht             0
leeftijd             0
herkomst             0
betwerk              0
onbbez               0
opleiding            0
hhgestinkg           0
oprijbewijsau        0
hhauto               0
brandstofpa1         0
brandstofepa1        0
brandstofpa2         0
brandstofepa2        0
hhefiets             0
ovstkaart            0
weekdag              0
feestdag             0
verplid              0
doel                 0
kmotiefv             0
vertpc               0
aankpc               0
khvm                 0
verttijd             0
aanktijd             0
choice_dur           0
choice_dist          0
bike_dur          5328
bike_dist         5328
car_dur          12375
car_dist         12375
pt_dur     

In [6]:
# fill in the missing values for the activity duration (actduur) by using the feature's median value derived from trips with the same purpose

data_2018_2019_final.loc[(pd.isna(data_2018_2019_final.actduur) & (data_2018_2019_final.doel == 1)),'actduur'] = \
                          data_2018_2019_final.loc[data_2018_2019_final.doel == 1,:].actduur.median()
data_2018_2019_final.loc[(pd.isna(data_2018_2019_final.actduur) & (data_2018_2019_final.doel == 2)),'actduur'] = \
                          data_2018_2019_final.loc[data_2018_2019_final.doel == 2,:].actduur.median()
data_2018_2019_final.loc[(pd.isna(data_2018_2019_final.actduur) & (data_2018_2019_final.doel == 3)),'actduur'] = \
                          data_2018_2019_final.loc[data_2018_2019_final.doel == 3,:].actduur.median()
data_2018_2019_final.loc[(pd.isna(data_2018_2019_final.actduur) & (data_2018_2019_final.doel == 4)),'actduur'] = \
                          data_2018_2019_final.loc[data_2018_2019_final.doel == 4,:].actduur.median()
data_2018_2019_final.loc[(pd.isna(data_2018_2019_final.actduur) & (data_2018_2019_final.doel == 5)),'actduur'] = \
                          data_2018_2019_final.loc[data_2018_2019_final.doel == 5,:].actduur.median()
data_2018_2019_final.loc[(pd.isna(data_2018_2019_final.actduur) & (data_2018_2019_final.doel == 6)),'actduur'] = \
                          data_2018_2019_final.loc[data_2018_2019_final.doel == 6,:].actduur.median()
data_2018_2019_final.loc[(pd.isna(data_2018_2019_final.actduur) & (data_2018_2019_final.doel == 7)),'actduur'] = \
                          data_2018_2019_final.loc[data_2018_2019_final.doel == 7,:].actduur.median()
data_2018_2019_final.loc[(pd.isna(data_2018_2019_final.actduur) & (data_2018_2019_final.doel == 8)),'actduur'] = \
                          data_2018_2019_final.loc[data_2018_2019_final.doel == 8,:].actduur.median()
data_2018_2019_final.loc[(pd.isna(data_2018_2019_final.actduur) & (data_2018_2019_final.doel == 9)),'actduur'] = \
                          data_2018_2019_final.loc[data_2018_2019_final.doel == 9,:].actduur.median()
data_2018_2019_final.loc[(pd.isna(data_2018_2019_final.actduur) & (data_2018_2019_final.doel == 10)),'actduur'] = \
                          data_2018_2019_final.loc[data_2018_2019_final.doel == 10,:].actduur.median()
data_2018_2019_final.loc[(pd.isna(data_2018_2019_final.actduur) & (data_2018_2019_final.doel == 11)),'actduur'] = \
                          data_2018_2019_final.loc[data_2018_2019_final.doel == 11,:].actduur.median()
data_2018_2019_final.loc[(pd.isna(data_2018_2019_final.actduur) & (data_2018_2019_final.doel == 12)),'actduur'] = \
                          data_2018_2019_final.loc[data_2018_2019_final.doel == 12,:].actduur.median()

data_2018_2019_final.loc[(pd.isna(data_2018_2019_final.actduur) & (data_2018_2019_final.doel == 13)),'actduur'] = \
                          data_2018_2019_final.loc[data_2018_2019_final.doel == 13,:].actduur.median()
data_2018_2019_final.loc[(pd.isna(data_2018_2019_final.actduur) & (data_2018_2019_final.doel == 14)),'actduur'] = \
                          data_2018_2019_final.loc[data_2018_2019_final.doel == 14,:].actduur.median()

In [7]:
# merge mode categories 3 (train) and 4 (bus/tram/metro) under 'transit' category (label 3)
data_2018_2019_final.loc[data_2018_2019_final.khvm == 4,'khvm'] = 3
data_2018_2019_final.loc[data_2018_2019_final.khvm == 5,'khvm'] = 4
data_2018_2019_final.loc[data_2018_2019_final.khvm == 6,'khvm'] = 5

In [8]:
# encode features

# age (age<18, 18<=age<=64, age>65)
data_2018_2019_final.leeftijd.dtype
data_2018_2019_final.loc[data_2018_2019_final.leeftijd <= 17,'leeftijd'] = 1
data_2018_2019_final.loc[((data_2018_2019_final.leeftijd >= 18) & (data_2018_2019_final.leeftijd <= 64)),'leeftijd'] = 2
data_2018_2019_final.loc[data_2018_2019_final.leeftijd >= 65, 'leeftijd'] = 3

# hhgestinkg (income) 
data_2018_2019_final.loc[(data_2018_2019_final.hhgestinkg < 5),'hhgestinkg'] = 1 # low income
data_2018_2019_final.loc[(data_2018_2019_final.hhgestinkg == 11),'hhgestinkg'] = 4 # unknown income values
data_2018_2019_final.loc[((data_2018_2019_final.hhgestinkg != 1) & (data_2018_2019_final.hhgestinkg != 4)),'hhgestinkg'] = 2 # high income

# departure and arrival times (1:morning, 2:afternoon, 3:evening)
data_2018_2019_final.loc[(data_2018_2019_final.verttijd < str(12)),'period_o'] = 1
data_2018_2019_final.loc[((data_2018_2019_final.verttijd >= str(12)) & (data_2018_2019_final.verttijd < str(18))),'period_o'] = 2
data_2018_2019_final.loc[((data_2018_2019_final.verttijd >= str(18)) & (data_2018_2019_final.verttijd < str(4))), 'period_o'] = 3

# weekdag (0: weekend and 1: weekday)
data_2018_2019_final.loc[((data_2018_2019_final.weekdag == 1) | (data_2018_2019_final.weekdag == 7)), 'weekdag'] = 0
data_2018_2019_final.loc[((data_2018_2019_final.weekdag != 1) & (data_2018_2019_final.weekdag != 7)), 'weekdag'] = 1

# hhauto (0: no car, 1: car, category 10: is unknown)
data_2018_2019_final.loc[((data_2018_2019_final.hhauto != 0) & \
                 (data_2018_2019_final.hhauto != 10)),'hhauto'] = 1

# hhpers (1: single-person household, 0: non single-person household)
data_2018_2019_final.loc[(data_2018_2019_final.hhpers != 1),'hhpers'] = 0

# betwerk (category 4 is unknown, 0: no paid-work, 1: paid-work)
data_2018_2019_final.loc[data_2018_2019_final['betwerk'] == 5, 'betwerk'] = 0
data_2018_2019_final.loc[((data_2018_2019_final['betwerk'] != 0)), 'betwerk'] = 1

# hhsam [categories 3, 4, 6, 7 correspond to 1 (children), while the remaining categories correspond to 0 (no children)].
data_2018_2019_final.loc[((data_2018_2019_final.hhsam != 3) & (data_2018_2019_final.hhsam != 4) &  (data_2018_2019_final.hhsam != 6) &  (data_2018_2019_final.hhsam != 7)),'hhsam'] = 0 
data_2018_2019_final.loc[data_2018_2019_final.hhsam != 0, 'hhsam'] = 1

# gemgr 
data_2018_2019_final.loc[((data_2018_2019_final.gemgr == 1)|(data_2018_2019_final.gemgr == 2)|(data_2018_2019_final.gemgr == 3)|\
                 (data_2018_2019_final.gemgr == 4)),'gemgr'] = 1
data_2018_2019_final.loc[((data_2018_2019_final.gemgr == 5)|(data_2018_2019_final.gemgr == 6)),'gemgr'] = 2
data_2018_2019_final.loc[((data_2018_2019_final.gemgr != 1) & (data_2018_2019_final.gemgr != 2)) ,'gemgr'] = 3

# kmotiefv (1: commute, 2: business, 3: other)
data_2018_2019_final.loc[((data_2018_2019_final.kmotiefv != 1)&(data_2018_2019_final.kmotiefv != 2)), 'kmotiefv'] = 3 #other

# ovstkaart (category 4: unknown, unknown category  is replaced with 0, since according to survey explanations respondent 
# was either younger than 15 or older than 40 years old)

data_2018_2019_final.loc[data_2018_2019_final.ovstkaart == 4, 'ovstkaart'] = 0


In [9]:
# missing values

percent_missing = data_2018_2019_final.isnull().sum() * 100 / len(data_2018_2019_final)
missing_values = pd.DataFrame({'column_name': data_2018_2019_final.columns,
                                 'percent_missing': percent_missing})

# percentage of missing values in each column
# due to a significant number of missing values for the walking trips (approximately 30%), we chose to exclude the walking class entirely from our analysis.
missing_values.loc[missing_values.percent_missing != 0,:]

,column_name,percent_missing
bike_dur,bike_dur,2.792336
bike_dist,bike_dist,2.792336
car_dur,car_dur,6.485577
car_dist,car_dist,6.485577
pt_dur,pt_dur,15.048111
pt_dist,pt_dist,15.048111
walk_dur,walk_dur,29.894449
walk_dist,walk_dist,29.894449
car_changes,car_changes,6.485577
pt_changes,pt_changes,15.048111


In [12]:
# remove respondents who have chosen car as their means of transport but do not poccess a driving license

index_remove = data_2018_2019_final.loc\
                     [((data_2018_2019_final.khvm == 1) & (data_2018_2019_final.oprijbewijsau == 0)),:].index

data_2018_2019_final.drop(index_remove, inplace = True)

# drop the unused features
data_2018_2019_final.drop(columns = ['opid','wopc','wogem','sted','prov','verplid','vertpc', 'aankpc','hhlft1', 'hhlft2', 
                                     'hhlft3', 'hhlft4','brandstofpa1', 'brandstofepa1', 'verttijd', 'aanktijd',
                                     'brandstofpa2', 'brandstofepa2', 'onbbez', 'walk_dur', 'walk_dist'], inplace = True)

In [13]:
#For every sample, an availability feature is generated for each mode, taking either the value 0 or 1. This feature indicates the mode's availability for 
# the particular trip. If the OTP software gives NaN for a mode then its availability is set equal to zero, otherwise it is set equal to 1.
# In the case of the car mode, the possession of a valid driving license is also considered.

data_2018_2019_final.loc[pd.isna(data_2018_2019_final.car_dur),'av_car'] = 0
data_2018_2019_final.loc[(data_2018_2019_final.oprijbewijsau == 0),'av_car'] = 0
data_2018_2019_final.loc[data_2018_2019_final.av_car != 0, 'av_car'] = 1

data_2018_2019_final.loc[pd.isna(data_2018_2019_final.pt_dur),'av_pt'] = 0
data_2018_2019_final.loc[data_2018_2019_final.av_pt != 0, 'av_pt'] = 1

data_2018_2019_final.loc[pd.isna(data_2018_2019_final.bike_dur),'av_bike'] = 0
data_2018_2019_final.loc[data_2018_2019_final.av_bike != 0, 'av_bike'] = 1

In [14]:
y_train_dcm = data_2018_2019_final.khvm
X_train_dcm = data_2018_2019_final.loc[:, data_2018_2019_final.columns != 'khvm']

# create the transit cost feature
transit_rate = (0.147 + 0.166) / 2  # eur/km average value from Rotterdam and the Hague areas
transit_base = 0.96  # euros

X_train_dcm['transit_cost'] = transit_base + transit_rate * (X_train_dcm['pt_dist']/1000)

X_train_dcm.loc[((X_train_dcm.ovstkaart == 1) & (X_train_dcm.weekdag == 1)), "transit_cost"] = 0
X_train_dcm.loc[((X_train_dcm.ovstkaart == 2) & (X_train_dcm.weekdag == 0)), "transit_cost"] = 0

X_train_dcm.loc[X_train_dcm.ovstkaart == 2,'ovstkaart'] = 1

# create the car cost feature
car_cost_km = 0.34 # euros/km
X_train_dcm['car_cost'] = (car_cost_km * (X_train_dcm['car_dist']/1000))\
                                   + (round(X_train_dcm['actduur']/60,2) * X_train_dcm['AVG_CHARGE'])

In [15]:
data_MNL = X_train_dcm.join(y_train_dcm)

# convert trip duration from sec to min
data_MNL.loc[:,'car_dur'] = data_MNL['car_dur']/60
data_MNL.loc[:,'pt_dur'] = data_MNL['pt_dur']/60
data_MNL.loc[:,'bike_dur'] = data_MNL['bike_dur']/60


# remove the samples for the OpenTripPlanner gives NaN for the chosen alternative 
ind_car_remove = data_MNL.loc[data_MNL.khvm == 1 & pd.isna(data_MNL.car_dur),:].index
data_MNL.drop(ind_car_remove,inplace = True)

ind_transit_remove = data_MNL.loc[((data_MNL.khvm == 3) & pd.isna(data_MNL.pt_dur)),:].index
data_MNL.drop(ind_transit_remove,inplace = True)

ind_bike_remove = data_MNL.loc[(data_MNL.khvm == 4) & (pd.isna(data_MNL.bike_dur)),:].index
data_MNL.drop(ind_bike_remove, inplace = True)

# remove the samples with zero transit costs
data_MNL.loc[data_MNL.transit_cost == 0,:].khvm.value_counts()
ind_remove = data_MNL.loc[data_MNL.transit_cost == 0,:].index
data_MNL.drop(ind_remove, inplace = True)

In [16]:
# imbalance ratio between the car (majority) class and the transit (minority) class
round(data_MNL.khvm.value_counts()[1]/data_MNL.khvm.value_counts()[3],2)

6.95

In [17]:
# save the data
data_MNL = data_MNL.loc[(data_MNL.khvm == 1) | (data_MNL.khvm == 3) | (data_MNL.khvm == 4)]
data_MNL.to_csv('./data/data_MNL.csv')